In [ ]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, JLD2, NetCDF

function qgprob(n::Int64)

    #All possible phenotypes
    pheno= collect(1:(2*n+1)) ./ (2*n+1)
    nt=length(pheno)

    G=zeros(Float64,n+1,n+1,n+1)

    for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
                m=collect(0:min(j,k,i+j-k))
                G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
    end

    for k in 0:n
        G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
        for i1 in 0:n
            G[i1+1,i1+1,k+1] /= 2
        end
    end

    ind_haplR=zeros(Float64,2*n+1, 2*n+1)

    for k in 0:n
        for i in 0:n
             ind_haplR[1+i,1+k] = G[1+i,1,1+k]
            for j in 0:n
                ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
            end
        end
    end

    R=zeros(Float64,nt,nt,nt)

    for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
         R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .*
                             ind_haplR[1+j,1+q .- (0:q)])
    end

    return R
end


R=qgprob(114)

save("R.jld", "data", R)

In [ ]:
using Distributed

addprocs(11)

@everywhere begin

using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, JLD2


R=load("R.jld")["data"]
#Input data
#SST data
SSTs=DataFrame(rcp2_6=collect(range(start=0,stop=0.775, length=95)),
              rcp4_5=collect(range(start=0,stop=1.6, length=95)),
              rcp6_0=collect(range(start=0,stop=1.925, length=95)),
              rcp8_5=collect(range(start=0,stop=3.0, length=95)))

tsteps=size(SSTs)[1]

[ SSTs[:,i] += rand(Normal(0,0.1),size(SSTs)[1]) for i in 1:(size(SSTs)[2]) ]

SSTdat=zeros(Float64,5,size(SSTs)[1],2)
for i in 1:3
    SSTdat[i,:,1]=SSTs[:,1]
    SSTdat[i,:,2]=SSTs[:,1] .+ ((i-1)/2)
end

#parameters
reps=collect(1:10)
pas=collect(0.0:0.5:1.0)
ms=collect(0.0:0.05:0.5)
tds=collect(1:3)

pars=collect(Iterators.product(reps,pas,ms,tds))

#Declare the function to simulate population dynamics
function multipop(par::Tuple{Int64, Float64, Float64, Int64})
        
        pa=par[2]
        mmat=[0 par[3]; 0 0]
        SST=SSTdat[par[4],:,:]

        popsize=1000.0

        #subpopulations
        Ns=[popsize/3,popsize*2/3]
        
        #Growth rate
        r=repeat([1.38],length(Ns))
        
        #Carrying capacity
        K=1.2.*Ns
    
        tsteps=size(SST)[1]
    
        nloci=(size(R)[1]-1)/2
        pheno= collect(1:(2*nloci+1)) ./ (2*nloci+1)
    
        N=rand(Uniform(0,1.0),length(Ns),size(R)[1])
        Ng0= N ./ sum(eachcol(N))
        Np0= Ng0 .* Ns
        Ngen=deepcopy(Ng0)
        Np=deepcopy(Np0)

        #Set up optimal traits value change with time using SST data
        #Initial value is the population trait mean (this assumes that the local populations are
        #already adapted to the local temperature).

        topt0= [sum(pheno .* Np[x,:])/sum(Np[x,:]) for x in 1:size(Np)[1]]
    
        topts=topt0 .+ (1.2 .* SST')
    
        #precalculate selection pressure on phenotypes across time steps
        probsurv=zeros(Float64, tsteps, length(Ns), length(pheno))
    
        [probsurv[x,y,:]=exp.(-((topts[y,x] .-pheno) .^2) ./ 0.16) for x in 1:tsteps, y in 1:length(Ns) ]
    
        #Store results
        res=zeros(Float64,tsteps+1,length(Ns),length(pheno))
    
        res[1,:,:]=Np0
    
        for t in 1:tsteps
    
            if sum(floor.(Np))<1
                    break
    
            else
    
                #Selection event
    
                Np=Np .* (probsurv[t,:,:])
    
                #Reproduction event
    
                if(pa<1)
    
                    newgen=zeros(Float64,length(Ns),length(pheno))
    
                    for i in 1:size(Ngen)[1]
    
                        Ngen[i,:]=Np[i,:]/sum(Np[i,:])
    
                        probs=Ngen[i,:]*(Ngen[i,:]')
    
                        for j in 1:size(Ngen)[2]
    
                            newgen[i,j]=sum(probs.*R[:,:,j])
                        end
    
                        Np[i,:]=((pa*sum(Np[i,:])) .* Ngen[i,:]) + (((1-pa)*sum(Np[i,:])) .* newgen[i,:])
    
                    end
                end
    
                #Settlement
    
                for i in 1:size(Np)[1]
    
                    #Logistic growth to simulate crowding effect during settlement
                    Ngen[i,:]=Np[i,:]/sum(Np[i,:])
    
                    Np[i,:]=(sum(Np[i,:])+ (r[i]*sum(Np[i,:])*(1-(sum(Np[i,:])/K[i])))) .* Ngen[i,:]
    
                end
    
                #Migration
                #add fraction of populations to each other (uniformly across all phenotypes) according to
                #the migration matrix mmat
                for i in 1:size(Np)[1], j in 1:size(Np)[1]
    
                    Np[i,:] += mmat[i,j] .* Np[j,:]
    
                end
    
                Np[Np .<1 ] .= 0
    
                res[t+1,:,:]=Np
    
            end
        end
    
        Ndat=[sum(res[x,:,:]) for x in 1:size(res)[1]]
    
        hl=findfirst(x-> x< (Ndat[1]/2), Ndat)
    
        resdat=DataFrame(rep=par[1],pa=par[2],m=par[3],td=par[4],hl=hl)
    
        CSV.write(string("C:\\Users\\mihir\\Documents\\coral_popgen\\codes\\res\\",par[1],"_",pa,"_",par[3],"_",par[4],".csv"),resdat)

end


end

        


In [ ]:
pmap(multipop,pars)